&nbsp;

# 14. Construction de l'état réduit 

---

&nbsp;

## 1. Objectif et positionnement


> #### Quel est l'état à retenir ?

Rappelons en condensé ce qui a été énoncé dans le notebook 10.

En l'état, nous démarrons le pré-traitement avec un champs spatial de SST qui évolue dans le temps avec des milliers de points spatiaux corrélés entre eux. 

La réduction de dimensionnalité est nécessaire pour clarifier ce qu'on veut faire apprendre à notre futur modèle. Je cite la partie 1. du notebook 10 : "Sans cela, on s'expose à un apprentissage instable, une sur-paramétrisation massive et une interprétation impossible. Le modèle ne doit apprendre ni le bruit, ni la grille, mais la dynamique.".

La nature géophysique des champs avec lesquels nous travaillons nous avait amené à penser que l'EOF serait la méthode adaptée pour réduire notre série d'anomalies de SST désaisonnalisée. On transforme ainsi notre matrice actuelle de la manière suivante :

$$ X(x, y, t) \approx \overset{K}{\underset{k=1}{\sum}}a_k(t)\phi_k(x,y) $$

Nous permettant d'obtenir le vecteur d'état composait des amplitudes de l'EOF de l'équation précédente, soit :

$$
\dot{a}_k(t) = f(\begin{pmatrix} a_1(t) \\ a_2(t) \\ \vdots \\ a_k(t) \end{pmatrix})
$$

Finalement, on fournit l'état dynamique réduit suivant au modèle SciML :

$$
\dot{a}_k(t) = \begin{pmatrix} \dot{a}_k(t) \\ \dot{a}_2(t) \\ \vdots \\ \dot{a}_1(t) \end{pmatrix} = f_\theta(a_1(t), a_2(t),..., a_k(t))
$$

&nbsp;

---

&nbsp;

## 2. Construction de l’état réduit par EOF

Commençons dès maintenant par réduire notre matrice étant donné que nous avions fait les vérifications dans l'analyse multivariée (voir Notebook 10).


In [3]:
import numpy as np
import xarray as xr
from sklearn.decomposition import PCA

# Load deseasonalized SST data
ds = xr.open_dataset("data/processed/sstDeseasonalizedCOPERNICUS20102019.nc")

# Extract SST data and reshape for PCA
sst = ds["analysed_sst"]
sstStacked = sst.stack(space=("latitude", "longitude"))
sstStacked = sstStacked.dropna("space")

X = sstStacked.values  # shape (Nt, Nspace)

K = 150  # Number of principal components to retain plus some margin if we want to test more later

pca = PCA(n_components=K, svd_solver="full")
PCs = pca.fit_transform(X)        # a_k(t)
EOFs = pca.components_            # phi_k(x)
explainedVar = pca.explained_variance_ratio_
